In [1]:
import json
import pandas as pd

# Загрузка JSON из файла
with open('C:\\Users\\Admin\\Downloads\\result.json', 'r', encoding='utf-8') as file:
    parsed_data = json.load(file)

# Извлечение данных о канале
channel_data = {
    "name": parsed_data["name"],
    "type": parsed_data["type"],
    "id": parsed_data["id"]
}

# Извлечение данных о сообщениях
messages = parsed_data["messages"]

# Создание списка для хранения данных
data = []

# Обработка каждого сообщения
for message in messages:
    # Базовые данные сообщения
    message_data = {
        "message_id": message["id"],
        "message_type": message["type"],
        "date": message["date"],
        "date_unixtime": message["date_unixtime"],
        "from": message.get("from", "Не указано"),
        "from_id": message.get("from_id", "Не указано")
    }

    # Обработка текста сообщения (если есть)
    if "text" in message:
        # Если текст — это список, объединяем его в строку
        if isinstance(message["text"], list):
            text_parts = []
            for part in message["text"]:
                if isinstance(part, dict):
                    text_parts.append(part.get("text", ""))
                else:
                    text_parts.append(str(part))
            message_data["text"] = " ".join(text_parts)
        else:
            message_data["text"] = message["text"]

    # Обработка реакций (если есть)
    if "reactions" in message:
        reactions = []
        for reaction in message.get("reactions", []):
            emoji = reaction.get("emoji", "Не указано")  # Используем .get(), чтобы избежать ошибки
            count = reaction.get("count", 0)  # Если "count" нет, возвращаем 0
            reactions.append(f"{emoji} ({count})")
        message_data["reactions"] = ", ".join(reactions)

    # Добавление данных в общий список
    data.append({**channel_data, **message_data})

    
    
# Создание DataFrame
df = pd.DataFrame(data)

# Сохранение в CSV
df.to_csv(r"C:\Users\Admin\Downloads\report_2025-04-03 09_55_25.962602\output.csv", index=False, encoding='utf-8')

# Вывод первых строк DataFrame
df.head()

,name,type,id,message_id,message_type,date,date_unixtime,from,from_id,text,reactions
0,💬 Data Practicum Chat,private_supergroup,1379846874,266690,service,2025-02-01T22:40:19,1738438819,Не указано,Не указано,,NaN
1,💬 Data Practicum Chat,private_supergroup,1379846874,266691,service,2025-02-02T21:59:34,1738522774,Не указано,Не указано,,NaN
2,💬 Data Practicum Chat,private_supergroup,1379846874,266693,message,2025-02-03T11:28:38,1738571318,Olga Varavina,user312724902,Всем большой привет! Приглашаю на свой уютный ...,NaN
3,💬 Data Practicum Chat,private_supergroup,1379846874,266694,message,2025-02-03T11:52:20,1738572740,Илья,user1349934990,А у тебя когда будет свой канал про аналитику?,NaN
4,💬 Data Practicum Chat,private_supergroup,1379846874,266695,message,2025-02-03T11:52:37,1738572757,Илья,user1349934990,Будешь туда голосовухи пятиминутные постить,😁 (4)


In [2]:
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from nltk.corpus import stopwords
import nltk

# Загрузка стоп-слов
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))  # Стоп-слова для русского языка

# Фильтрация и очистка текста
def clean_text(text):
    text = str(text).lower().strip()  # Приводим к нижнему регистру
    text = re.sub(r'\s+', ' ', text)  # Убираем лишние пробелы
    text = re.sub(r'[^a-zA-Zа-яА-Я0-9 ]', '', text)  # Убираем пунктуацию
    words = text.split()
    text = ' '.join([word for word in words if word not in stop_words])  # Убираем стоп-слова
    return text

df = df.dropna(subset=['text'])  # Удаляем NaN

df['clean_text'] = df['text'].apply(clean_text)
df = df[df['clean_text'].apply(lambda x: len(x.split()) > 1)]  # Оставляем только сообщения с 2+ словами

# Векторизация текста с TF-IDF
vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
X = vectorizer.fit_transform(df['clean_text'])

# Автоматическое определение количества кластеров методом локтя
distortions = []
K_range = range(2, min(10, len(df)))  # Число кластеров не может превышать число сообщений

for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X)
    distortions.append(sum(np.min(cdist(X.toarray(), km.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

optimal_k = K_range[np.argmin(distortions)]  # Выбираем "локоть"

# Кластеризация KMeans
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df['cluster'] = kmeans.fit_predict(X)

# Функция для выбора ключевых предложений из кластера
def get_key_sentences(cluster):
    cluster_indices = np.where(df['cluster'] == cluster)[0]
    if len(cluster_indices) == 0:
        return []
    cluster_vectors = X[cluster_indices]
    centroid = kmeans.cluster_centers_[cluster].reshape(1, -1)
    distances = cdist(cluster_vectors.toarray(), centroid, 'euclidean')
    closest_idx = cluster_indices[np.argmin(distances)]
    return [df.iloc[closest_idx]['text']]



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# 1. Добавим размер кластера (cluster_size)
cluster_counts = df['cluster'].value_counts().to_dict()
df['cluster_size'] = df['cluster'].map(cluster_counts)

# 2. Извлекаем ключевые слова для каждого кластера
def get_top_keywords_per_cluster(tfidf_matrix, labels, features, top_n=10):
    df_keywords = {}
    for cluster_num in np.unique(labels):
        indices = np.where(labels == cluster_num)[0]
        mean_tfidf = tfidf_matrix[indices].mean(axis=0).A1
        top_indices = mean_tfidf.argsort()[::-1][:top_n]
        top_keywords = [features[i] for i in top_indices]
        df_keywords[cluster_num] = ', '.join(top_keywords)
    return df_keywords

feature_names = vectorizer.get_feature_names_out()
keywords_dict = get_top_keywords_per_cluster(X, df['cluster'].values, feature_names)
df['keywords'] = df['cluster'].map(keywords_dict)

# 3. Добавим ключевое предложение
key_sentences_dict = {}
for cluster in range(optimal_k):
    sentences = get_key_sentences(cluster)
    key_sentences_dict[cluster] = sentences[0] if sentences else None
df['key_sentence'] = df['cluster'].map(key_sentences_dict)

# 4. Готовый финальный фрейм
df[['name', 'text', 'clean_text', 'cluster', 'cluster_size', 'keywords', 'key_sentence']].head(20)

,name,text,clean_text,cluster,cluster_size,keywords,key_sentence
2,💬 Data Practicum Chat,Всем большой привет! Приглашаю на свой уютный ...,всем большой привет приглашаю свой уютный кана...,1,562,"это, просто, тебе, сегодня, nda, знаю, ещ, мог...",начала борьбу за призы)
3,💬 Data Practicum Chat,А у тебя когда будет свой канал про аналитику?,свой канал аналитику,1,562,"это, просто, тебе, сегодня, nda, знаю, ещ, мог...",начала борьбу за призы)
4,💬 Data Practicum Chat,Будешь туда голосовухи пятиминутные постить,будешь туда голосовухи пятиминутные постить,1,562,"это, просто, тебе, сегодня, nda, знаю, ещ, мог...",начала борьбу за призы)
5,💬 Data Practicum Chat,"Потому что сделаны так, будто устарели уже лет...",сделаны устарели лет цать особенно кажется пов...,1,562,"это, просто, тебе, сегодня, nda, знаю, ещ, мог...",начала борьбу за призы)
7,💬 Data Practicum Chat,"Не, это не так раздражает",это раздражает,5,26,"это, чат, считается, вс, понастоящему, тип, та...","Не, это не так раздражает"
8,💬 Data Practicum Chat,"Нужны голосовуки с эээ, нууу, и 10 секундными ...",нужны голосовуки эээ нууу 10 секундными паузами,1,562,"это, просто, тебе, сегодня, nda, знаю, ещ, мог...",начала борьбу за призы)
9,💬 Data Practicum Chat,"Звонили из ада, просили передать что ждут на м...",звонили ада просили передать ждут мастеркласс,1,562,"это, просто, тебе, сегодня, nda, знаю, ещ, мог...",начала борьбу за призы)
10,💬 Data Practicum Chat,"Ты хотел сказать ""ждут мастер-класс""?",хотел сказать ждут мастеркласс,1,562,"это, просто, тебе, сегодня, nda, знаю, ещ, мог...",начала борьбу за призы)
11,💬 Data Practicum Chat,"Их сделали, чтобы понравится человеку, котором...",сделали понравится человеку которому отчитываю...,1,562,"это, просто, тебе, сегодня, nda, знаю, ещ, мог...",начала борьбу за призы)
12,💬 Data Practicum Chat,"Благодарствую, ещё не проснулся",благодарствую ещ проснулся,1,562,"это, просто, тебе, сегодня, nda, знаю, ещ, мог...",начала борьбу за призы)
